## FASE II

Se presenta a continuación el desarrollo de la actividad correspondiente a la fase II.

Considera que cada producto que se encuentra en el almacén central, además de ser identificado por el tipo de embalaje, también posee como propiedad sus dimensiones (largo, ancho y altura con un máximo de 100,0 cm y un mínimo de 5,0 cm en cada dimensión), peso (kg), el tipo de manipulación que requiere (frágil, normal), lugar de procedencia (A, B, C, D) y temperatura de almacenamiento (ambiente, refrigerado).

Si el vehículo autónomo tiene integrados dos protocolos de manipulación de carga (Protocolo_1 y Protocolo_2), realiza las siguientes operaciones:
1. Genera un conjunto de datos de prueba (mínimo 10,000 muestras), considerando los criterios antes indicados y teniendo en cuenta tu experiencia personal, donde, a partir del análisis de la información del producto, estos puedan ser catalogados dentro los protocolos disponibles.

2. Construye un modelo de aprendizaje automático supervisado a partir del conjunto de datos elaborado, que sea capaz de determinar el tipo de protocolo que debe seguir el vehículo al manipular un producto cualquiera en el trayecto del almacén a la zona de manufactura.


Importamos las librerías requeridas para ejecutar los requerimientos de los puntos anteriores.

* La librería "svd" se encontró en un tutorial de internet relacionado a la generación de datos sintéticos.
[Top 3 Python Packages for Generating Synthetic Data](https://towardsdatascience.com/top-3-python-packages-to-generate-synthetic-data-33a351a5de0c)
* Para esta generación de datos se requiere una pequeña muestra con datos de ejemplo así que se generó un archivo csv llamado "paquetes_fase_II.csv" con una muestra de 10 registros que usa la librería svd para general los datos sintéticos de 10000.

In [18]:
from sdv.tabular import GaussianCopula #Librería usada para apoyar la creación de datos sintéticos.
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn import metrics
import numpy as np

In [19]:
data = pd.read_csv('https://raw.githubusercontent.com/janus78/MasterIA-Notebooks_Tetra3/master/paquetes_fase_II.csv')
data.head()

,Embalaje,Ancho(cm),Largo(cm),Alto(cm),Procedencia,Manipulacion,Temperatura,Protocolo
0,A,35,40,50,A,normal,ambiente,protocolo_1
1,B,80,70,60,B,fragil,refrigerado,protocolo_2
2,C,5,8,4,C,fragil,ambiente,protocolo_2
3,A,25,16,24,B,normal,ambiente,protocolo_1
4,C,12,21,5,A,fragil,refrigerado,protocolo_1


In [20]:
model = GaussianCopula() #Se crea una instancia de la librería "GaussianCopula" para generar datos sintéticos.
model.fit(data) #Se entrena el modelo con los datos de ejemplo.

In [21]:
sample = model.sample(10000) #Se generan datos sintéticos, un total de 10000.
sample.head() #Se muestran los primeros 5 registros.

,Embalaje,Ancho(cm),Largo(cm),Alto(cm),Procedencia,Manipulacion,Temperatura,Protocolo
0,C,49,44,53,C,normal,ambiente,protocolo_2
1,A,96,76,85,A,normal,ambiente,protocolo_2
2,A,56,56,57,B,fragil,refrigerado,protocolo_2
3,B,75,54,55,B,fragil,ambiente,protocolo_2
4,C,5,8,5,C,normal,refrigerado,protocolo_1


In [22]:
# se valida el tamaño de la muestra generada.
print(sample.shape)
print(sample.isna().sum()) #Se verifica que no haya nulos.

(10000, 8)
Embalaje        0
Ancho(cm)       0
Largo(cm)       0
Alto(cm)        0
Procedencia     0
Manipulacion    0
Temperatura     0
Protocolo       0
dtype: int64


In [23]:
from sdv.evaluation import evaluate #Librería usada para evaluar el modelo de generación de datos sintéticos.
evaluate(sample, data, metrics=['CSTest', 'KSTest'], aggregate = False) #Se evalúa el modelo con los datos de ejemplo generados con la librería.

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,CSTest,Chi-Squared,0.962569,0.962569,0.0,1.0,MAXIMIZE,None
1,KSTest,Inverted Kolmogorov-Smirnov D statistic,0.820559,0.820559,0.0,1.0,MAXIMIZE,None


Podemos notar en las métricas mostradas en la celda anterior que la calidad de la generación de datos sintéticos muestra buenos valores en la calidad de los mismos con score entre el 80 y 90%.

* A continuación se va a proceder a categorizar con enteros las columnas que tienen valores string ya que esto facilita la implementación del modelo de aprendizaje automático.

In [24]:
#Se usará el ordinal encoder de sklearn para categorizar las columnas que tienen valores string.
ord_enc = OrdinalEncoder()
sample['Manipulacion_enc'] = ord_enc.fit_transform(sample[['Manipulacion']]) #Se categoriza la columna "Manipulacion" con enteros.
sample['Procedencia_enc'] = ord_enc.fit_transform(sample[['Procedencia']])  #Se categoriza la columna "Procedencia" con enteros.
sample['Embalaje_enc'] = ord_enc.fit_transform(sample[['Embalaje']])    #Se categoriza la columna "Embalaje" con enteros.
sample['Temperatura_enc'] = ord_enc.fit_transform(sample[['Temperatura']])  #Se categoriza la columna "Temperatura" con enteros.
sample['Protocolo_enc'] = ord_enc.fit_transform(sample[['Protocolo']])  #Se categoriza la columna "Protocolo_1" con enteros.
sample.head()

,Embalaje,Ancho(cm),Largo(cm),Alto(cm),Procedencia,Manipulacion,Temperatura,Protocolo,Manipulacion_enc,Procedencia_enc,Embalaje_enc,Temperatura_enc,Protocolo_enc
0,C,49,44,53,C,normal,ambiente,protocolo_2,1.0,2.0,2.0,0.0,1.0
1,A,96,76,85,A,normal,ambiente,protocolo_2,1.0,0.0,0.0,0.0,1.0
2,A,56,56,57,B,fragil,refrigerado,protocolo_2,0.0,1.0,0.0,1.0,1.0
3,B,75,54,55,B,fragil,ambiente,protocolo_2,0.0,1.0,1.0,0.0,1.0
4,C,5,8,5,C,normal,refrigerado,protocolo_1,1.0,2.0,2.0,1.0,0.0


* Se procede a obtener el dataset final "X" con las columnas requeridas para la implementación del modelo de aprendizaje automático.
* Y se obtiene la columna de etiquetas "y" que corresponde al protocolo que debe seguir el vehículo al manipular un producto cualquiera en el trayecto del almacén a la zona de manufactura.

In [25]:
X = sample[['Manipulacion_enc', 'Procedencia_enc', 'Embalaje_enc', 'Temperatura_enc', 'Ancho(cm)', 'Largo(cm)', 'Alto(cm)']] #Se obtiene el dataset final "X" con las columnas requeridas para la implementación del modelo de aprendizaje automático.
y = sample['Protocolo_enc'] #Se obtiene la columna de etiquetas "y" que corresponde al protocolo que debe seguir el vehículo al manipular un producto cualquiera en el trayecto del almacén a la zona de manufactura.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) #Se separan los datos de entrenamiento y prueba.
X.head()

,Manipulacion_enc,Procedencia_enc,Embalaje_enc,Temperatura_enc,Ancho(cm),Largo(cm),Alto(cm)
0,1.0,2.0,2.0,0.0,49,44,53
1,1.0,0.0,0.0,0.0,96,76,85
2,0.0,1.0,0.0,1.0,56,56,57
3,0.0,1.0,1.0,0.0,75,54,55
4,1.0,2.0,2.0,1.0,5,8,5


* Se utiliza el clasificador Perceptron para implementar el modelo de aprendizaje automático.

In [26]:
clf = Perceptron(random_state=0) #Se crea una instancia de la clase Perceptron.
clf.fit(X_train, y_train) #Se entrena el modelo.

Perceptron()

In [27]:
y_pred = clf.predict(X_test) #Se predice el protocolo que debe seguir el vehículo al manipular un producto cualquiera en el trayecto del almacén a la zona de manufactura.
print(metrics.classification_report(y_test, y_pred)) #Se muestra el reporte de metricas del modelo.

              precision    recall  f1-score   support

         0.0       0.96      0.51      0.67       891
         1.0       0.71      0.98      0.83      1109

    accuracy                           0.77      2000
   macro avg       0.84      0.75      0.75      2000
weighted avg       0.82      0.77      0.76      2000



**Conclusiones:**
Las conclusiones de esta primera parte de la actividad son que podemos darnos cuenta de la importancia de tener una forma con la que generar datos de prueba de calidad y que para encontrarle fue necesario hacer una busqueda que tomó un tiempo considerable.
Además de esto el trabajo usual de realizar el análisis, ajuste y limpieza de datos que tambien toma buena parte del tiempo.
Fue muy ilustrativo realizar esta parte del ejercicioya que nos obligó a buscar nuevas formas de realizar las cosas que se requieren para la actividad.

## Parte II: Pregunta 5 ##
Esta parte corresponde a la pregunta 5 de la actividad. que es relacionada a la distribución de los productos en la planta de manufactura.

1. La planta industrial tiene una distribución como la que se muestra en la figura 1.

   ![Mapeo almacen](https://raw.githubusercontent.com/janus78/MasterIA-Notebooks_Tetra3/master/f2.png)

3. Cada vehículo puede transportar un máximo de tres productos al mismo tiempo.
4. Los productos en el almacén no poseen un orden determinado.
5. Un vehículo puede seleccionar cualquier producto con la misma probabilidad.
6. Elabora un programa que sea capaz de determinar la ruta óptima que debe tomar el vehículo para distribuir los productos por la planta industrial, garantizando que:

7. Nunca repita dos veces el mismo destino en cada viaje.
8. Los productos del protocolo_1 tengan prioridad.
9. La ruta trazada sea siempre la menor.

Para el desarrollo de este ejercicio se ha realizado mucha busqueda, revisión y prácticas de diferentes tutoriales esperando lograr una implementación adecuada. Aún quedan muchas dudas y la inseguridad de que sea correcta la implementación.
A continuación se presenta el desarrollo de este ejercicio.

Dentro de muchos de los tutoriales, guías y consejos que se han revisado se notaba la incidencia de la creacón y uso de una matriz de distancias por lo que es la aproximación que se utiliza en este caso.

In [28]:
#Importando las librerias necesarias.
from scipy.spatial import distance_matrix #Se importa la libreria de distancia.

In [29]:
#Se crea un dataframe que se va a usar para la creación de la matriz de distancias.
puntos_entrega = [[0, 300], [400, 0], [700, 0], [1000, 300], [1000, 200], [100, 600], [400, 600]] #Se crea una matriz de puntos de entrega.
nombre_puntos = ['O', 'B2', 'B1', 'C2', 'C1', 'A1', 'A2'] #Se crea una lista de nombres de puntos.
df_matriz = pd.DataFrame(puntos_entrega, columns=['x', 'y'], index=nombre_puntos) #Se crea un dataframe con los puntos de entrega.
df_matriz.head(6) #Se muestra el dataframe.

,x,y
O,0,300
B2,400,0
B1,700,0
C2,1000,300
C1,1000,200
A1,100,600


In [30]:
#Se crea un dataframe con la matriz de distancias.
df_matriz_dist = pd.DataFrame(distance_matrix(df_matriz.values, df_matriz.values), index=df_matriz.index, columns=df_matriz.index).astype(int) #Se crea un dataframe con la matriz de distancias.
df_matriz_dist.head(7) #Se muestra el dataframe.

,O,B2,B1,C2,C1,A1,A2
O,0,500,761,1000,1004,316,500
B2,500,0,300,670,632,670,600
B1,761,300,0,424,360,848,670
C2,1000,670,424,0,100,948,670
C1,1004,632,360,100,0,984,721
A1,316,670,848,948,984,0,300
A2,500,600,670,670,721,300,0


In [31]:
#Se obtiene una predicción del total de productos en el dataset sintetico.
y_all = clf.predict(X) #Se obtiene la predicción de los protocolos.

In [32]:
X_full = X.copy() #Se crea una copia del dataset original.
X_full['Protocolo_enc'] = y_all #Se agrega la columna de etiquetas al dataset.
X_full.head()

,Manipulacion_enc,Procedencia_enc,Embalaje_enc,Temperatura_enc,Ancho(cm),Largo(cm),Alto(cm),Protocolo_enc
0,1.0,2.0,2.0,0.0,49,44,53,1.0
1,1.0,0.0,0.0,0.0,96,76,85,1.0
2,0.0,1.0,0.0,1.0,56,56,57,1.0
3,0.0,1.0,1.0,0.0,75,54,55,1.0
4,1.0,2.0,2.0,1.0,5,8,5,0.0


In [33]:
#Se obtienen los totales de productos para cada protocolo y embalaje.
total_pt1_A = X_full.apply(lambda x : x['Protocolo_enc'] == 0 and x['Embalaje_enc'] == 0, axis=1).sum() #Se obtiene el total de productos para el protocolo_1 y el embalaje A.
total_pt2_A = X_full.apply(lambda x : x['Protocolo_enc'] == 1 and x['Embalaje_enc'] == 0, axis=1).sum() #Se obtiene el total de productos para el protocolo_2 y el embalaje A.
total_pt1_B = X_full.apply(lambda x : x['Protocolo_enc'] == 0 and x['Embalaje_enc'] == 1, axis=1).sum() #Se obtiene el total de productos para el protocolo_1 y el embalaje B.
total_pt2_B = X_full.apply(lambda x : x['Protocolo_enc'] == 1 and x['Embalaje_enc'] == 1, axis=1).sum() #Se obtiene el total de productos para el protocolo_2 y el embalaje B.
total_pt1_C = X_full.apply(lambda x : x['Protocolo_enc'] == 0 and x['Embalaje_enc'] == 2, axis=1).sum() #Se obtiene el total de productos para el protocolo_1 y el embalaje C.
total_pt2_C = X_full.apply(lambda x : x['Protocolo_enc'] == 1 and x['Embalaje_enc'] == 2, axis=1).sum() #Se obtiene el total de productos para el protocolo_2 y el embalaje C.

print(f"Los totales por embalaje y protocolo son:\nEmbalaje A, Protocolo 1: {total_pt1_A}\nEmbalaje A, Protocolo 2: {total_pt2_A}\nEmbalaje B, Protocolo 1: {total_pt1_B}\nEmbalaje B, Protocolo 2: {total_pt2_B}\nEmbalaje C, Protocolo 1: {total_pt1_C}\nEmbalaje C, Protocolo 2: {total_pt2_C}") #Se muestra el total de productos para cada protocolo y embalaje.

Los totales por embalaje y protocolo son:
Embalaje A, Protocolo 1: 1346
Embalaje A, Protocolo 2: 2987
Embalaje B, Protocolo 1: 877
Embalaje B, Protocolo 2: 2955
Embalaje C, Protocolo 1: 84
Embalaje C, Protocolo 2: 1751


Se pudieron obtener los totales de productos para cada protocolo y embalaje. Debido a restricciones que se encontraron con la herramienta que con eltiempo no alcancé a encontrar como solucionar no se usaron estos paquetes ya que la implementación de este algoritmo no se pudo lograr. Tiene una restricción de que la canticas de paquetes a entregar no puede ser mayor a la capacidad del vehículo.
Traté de implementar un pickup and delivery pereo ya no alcancé a lograrlo.
Hice la mayor cantidad de intentos que el tiempo libre me permitió.

In [34]:
#Creando el arreglo de demanda
demanda = [total_pt2_B//100, total_pt1_B//100, total_pt2_C//100, total_pt1_C//100, total_pt1_A//100, total_pt2_A//100] #Se crea un arreglo de demanda.
demanda

[29, 8, 17, 0, 13, 29]

* Construyendo el modelo usando Google OR Tools.
OR Tools de Google es un set de herramientas para optimización combinatoria que se trata de encontrar la mejor solucion de un set de posibles soluciones.

In [36]:
#Se importan las librerias necesarias.
from ortools.constraint_solver import routing_enums_pb2 #Se importa la libreria de routing_enums_pb2.
from ortools.constraint_solver import pywrapcp #Se importa la libreria de pywrapcp.

In [37]:
#Se convierte la matiz de distancias a numpy.
matriz_dist = df_matriz_dist.to_numpy() #Se convierte la matiz de distancias a numpy.

En este caso no se pudo usar el valor original del total de paquetes por protocolo y embalaje por lo que el arreglo de demanda tiene valores que son iguales o menores a la capacidad del vehiculo que es de 3.

In [68]:
def crear_modelo_datos():
    datos = {}
    datos['matriz_dist'] = matriz_dist #Se agrega la matriz de distancias al diccionario.
    datos['demanda'] = [0, 1, 3, 1, 2, 1, 2] #Se agrega la demanda al diccionario.
    datos['capacidad_vehiculo'] = [3, 3, 3, 3] #Se agrega la capacidad del vehiculo al diccionario.
    datos['num_vehiculos'] = 4 #Se agrega el numero de vehiculos al diccionario.
    datos['almacen'] = 0 #Se agrega el almacen al diccionario.
    return datos #Se retorna el diccionario.

In [69]:
def imprimir_solucion(datos, route_manager, rutas, solucion):
    print(f'Objetivo: {solucion.ObjectiveValue()}')
    nodos_eliminados = 'Nodos eliminados:'
    for node in range(rutas.Size()):
        if rutas.IsStart(node) or rutas.IsEnd(node):
            continue
        if solucion.Value(rutas.NextVar(node)) == node:
            nodos_eliminados += ' {}'.format(route_manager.IndexToNode(node))
    print(nodos_eliminados)
    distancia_total = 0 #Se inicializa la distancia total.
    carga_total = 0 #Se inicializa la carga total.
    for id_vehiculo in range(datos['num_vehiculos']): #Para cada vehiculo.
        indice = rutas.Start(id_vehiculo) #Se obtiene el indice de inicio.
        plan_salida = f'Ruta para el vehiculo {id_vehiculo + 1}:\n' #Se crea el plan de salida.
        distancia_ruta = 0 #Se inicializa la distancia de la ruta.
        carga_ruta = 0 #Se inicializa la carga de la ruta.
        while not rutas.IsEnd(indice): #Mientras no se llegue al final de la ruta.
            indice_nodo = route_manager.IndexToNode(indice) #Se obtiene el indice del nodo.
            carga_ruta += datos['demanda'][indice_nodo] #Se agrega la cantidad de productos en la orden.
            plan_salida += '{0} paquetes({1}) ->'.format(indice_nodo, carga_ruta) #Se agrega el nodo al plan de salida.
            indice_previo = indice #Se guarda el indice anterior.
            indice = solucion.Value(rutas.NextVar(indice)) #Se obtiene el siguiente indice.
            distancia_ruta += rutas.GetArcCostForVehicle(indice_previo, indice, id_vehiculo) #Se agrega la distancia de la ruta.
        plan_salida += ' {0} paquetes({1})\n'.format(route_manager.IndexToNode(indice), carga_ruta) #Se agrega el nodo al plan de salida.
        plan_salida += 'Distancia de la ruta: {} m\n'.format(distancia_ruta) #Se agrega la distancia de la ruta.
        plan_salida += 'Paquetes entregados: {} (paquetes)\n'.format(carga_ruta) #Se agrega la cantidad de productos entregados.
        print(plan_salida) #Se imprime el plan de salida.
        distancia_total += distancia_ruta #Se agrega la distancia de la ruta.
        carga_total += carga_ruta #Se agrega la carga de la ruta.
    print('Distancia total: {} m'.format(distancia_total)) #Se imprime la distancia total.
    print('Paquetes entregados: {} (paquetes)'.format(carga_total)) #Se imprime la cantidad de productos entregados.

In [71]:
#Funcion principal
def main():
    datos = crear_modelo_datos() #Se crea el diccionario de datos.
    route_manager = pywrapcp.RoutingIndexManager(len(datos['matriz_dist']), datos['num_vehiculos'], datos['almacen']) #Se crea el manager de rutas.
    rutas = pywrapcp.RoutingModel(route_manager) #Se crea el modelo de rutas.

    print(type(rutas))

    def llamada_distancia(indice_desde, indice_hasta):
        nodo_desde = route_manager.IndexToNode(indice_desde) #Se obtiene el nodo desde.
        nodo_hasta = route_manager.IndexToNode(indice_hasta) #Se obtiene el nodo hasta.
        return datos['matriz_dist'][nodo_desde][nodo_hasta] #Se retorna la distancia entre los nodos.

    indice_callback_transito = rutas.RegisterTransitCallback(llamada_distancia) #Se registra el callback de transito.

    #Definiendo el costo de la ruta
    rutas.SetArcCostEvaluatorOfAllVehicles(indice_callback_transito) #Se define el costo de la ruta.

    #Calculando la cantidad de productos en la orden.
    def llamada_cantidad(indice_desde):
        nodo_desde = route_manager.IndexToNode(indice_desde) #Se obtiene el nodo desde.
        return datos['demanda'][nodo_desde] #Se retorna la cantidad de productos en la orden.

    #Agregando la restricción de capacidad
    indice_callback_demanda = rutas.RegisterUnaryTransitCallback(llamada_cantidad) #Se registra el callback de la demanda.

    rutas.AddDimensionWithVehicleCapacity(indice_callback_demanda, 0, datos['capacidad_vehiculo'], True, 'Capacity') #Se agrega la restriccion de capacidad.

    #Agregando codigo para permitir omitir nodos.
    penalty = 1000
    for node in range(1, len(datos['matriz_dist'])):
        rutas.AddDisjunction([route_manager.NodeToIndex(node)], penalty)


    #Obteniendo la primera solución heurística.
    parametros_busqueda = pywrapcp.DefaultRoutingSearchParameters() #Se crea el parametros de búsqueda.
    parametros_busqueda.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC) #Se define la estrategia de búsqueda.
    parametros_busqueda.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.AUTOMATIC) #Se define la metaheuristica de búsqueda.
    parametros_busqueda.time_limit.FromSeconds(20) #Se define el tiempo limite de búsqueda.

    #Obteniendo la solución.
    solucion = rutas.SolveWithParameters(parametros_busqueda) #Se obtiene la solución.

    #Imprimiendo la solución.
    if solucion:
        imprimir_solucion(datos, route_manager, rutas, solucion) #Se imprime la solución.
    else:
        print('No se pudo obtener una solución.') #Se imprime que no se pudo obtener una solución.

In [72]:
if __name__ == '__main__':
    main() #Se ejecuta la función principal.

<class 'ortools.constraint_solver.pywrapcp.RoutingModel'>
Objetivo: 5220
Nodos eliminados: 2
Ruta para el vehiculo 1:
0 paquetes(0) ->4 paquetes(2) ->3 paquetes(3) -> 0 paquetes(3)
Distancia de la ruta: 2104 m
Paquetes entregados: 3 (paquetes)

Ruta para el vehiculo 2:
0 paquetes(0) -> 0 paquetes(0)
Distancia de la ruta: 0 m
Paquetes entregados: 0 (paquetes)

Ruta para el vehiculo 3:
0 paquetes(0) ->1 paquetes(1) -> 0 paquetes(1)
Distancia de la ruta: 1000 m
Paquetes entregados: 1 (paquetes)

Ruta para el vehiculo 4:
0 paquetes(0) ->5 paquetes(1) ->6 paquetes(3) -> 0 paquetes(3)
Distancia de la ruta: 1116 m
Paquetes entregados: 3 (paquetes)

Distancia total: 4220 m
Paquetes entregados: 7 (paquetes)
